In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import random
from pprint import pprint

#ignore all warnings
import warnings
warnings.filterwarnings('ignore')

sns.set_style("darkgrid")

from helper_functions import determine_type_of_feature

# Given with df and test_size(porportion) as arguments, split the
# original df into two parts by randomly selecting indices 
def train_test_split(df, test_size):
    test_size = int(len(df) * test_size)

    # indices contain all the index values of the DF
    indices = df.index.tolist()

    # Chooses k unique random elements from a population sequence or set for test indices 
    test_indices = random.sample(population=indices, k = test_size)

    # loc[] allows us to acces certain rows
    test_df = df.loc[test_indices]

    # Create test_df by dropping test_indices(rows) from original DF
    train_df = df.drop(test_indices)

    return train_df, test_df
    
#=================================================================================

# returnign boolean value
# True if the data is pure, False otherwise

def check_purity(data):
    # Last Column All rows
    label_col = data[:,-1]
    # Checking unique values in the column
    unique_classes = np.unique(label_col)

    # If only single class is in the data, return True
    if len(unique_classes)==1:
        return True
    else:
        return False    


#=================================================================================

# classifying the class type (in string data type)of given data
# can customize the classification if the number of data point is less the k

def create_leaf(data, ml_task):
    
    label_col = data[:,-1]
    
    if ml_task == "regression":
        leaf = np.mean(label_col)
        
    #classification    
    else:    
        # Returning unique values with their individual counts 
        unique_classes, classes_count = np.unique(label_col,return_counts=True)

        # Index of largest class
        largest_class_index = np.argmax(classes_count)

        leaf = unique_classes[largest_class_index]
        # returning largest class    
    
    return leaf

#=================================================================================

# returning dictionary where the keys are indices of columns and 
# values are list of all the potential split

# Loop only the features in random_subspace
def get_potential_splits(data,random_subspace):
    
    potential_splits = {}
    _, n_columns = data.shape
    
    # exclude last column of data frame(label)
    
    column_indices = list(range(n_columns - 1))
    
    # if random subspace is defined 
    if random_subspace != None and random_subspace <= len(column_indices):
        column_indices = random.sample(population = column_indices, k = random_subspace)
    
    for column_index in column_indices:
        
        #select all the rows of particular column
        values = data[:, column_index]
        
        # getting unique values from list of values
        unique_values = np.unique(values)
        
        potential_splits[column_index] = unique_values
    
    return potential_splits

#=================================================================================

# Split data into sections
# Split is determine by name of the feature(column) and numerical value
# Returning two numpy 2D array: below and above the numerical split point \


def split_data(data, split_column, split_value):
   
    # creating mask for the split
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    
    if type_of_feature == "continuous":
        # getting only the rows(data) satisfying the condition
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
     
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

#=================================================================================

# For the regression task, we use MSE instead of entropy,
# it calculate the deviation of actual value from the mean 

def calculate_mse(data):
    actual_values = data[:, -1]
    
    # get_potential_split might return empty list(empty data)
    # the if case handle the case
    if len(actual_values) == 0:  
        mse = 0
        
    else:
        prediction = np.mean(actual_values)
        mse = np.mean((actual_values - prediction) **2)
    
    return mse

#=================================================================================

# Iterate through all the splits from potential split function and 
# return a split with lowest overall entropy
# -> need to calculate Entropy first follow by overall entropy

# calculate entropy 
def calculate_entropy(data):
    
    label_col = data[:,-1]
    # to get Pi(prob of class i), need to get the names of unique class and their count 
    _, counts = np.unique(label_col, return_counts=True)

    # probabilities of each class 
    probabilities = counts / counts.sum()

    # Calculating Entropy using element wise operation of numpy
    entropy = sum(probabilities * -np.log2(probabilities))
    
    return entropy
#=================================================================================

# calculate entropy of entire plot
def calculate_overall_metric(data_below, data_above, metric_function):

    # based on the equation above
    num_total_data_points = len(data_below) + len(data_above)

    p_data_below = len(data_below) / num_total_data_points
    p_data_above = len(data_above) / num_total_data_points

    overall_metric = ((p_data_below * metric_function(data_below)) +
                      (p_data_above * metric_function(data_above)))

    return overall_metric
    

#=================================================================================

# determine best split using data and potential split dictionary

def find_best_split(data, potential_splits, ml_task):
    '''
    returning best_split_col, best_split_val which will be use to split the data
    best_split_col: feature(column) resulting lowest entropy value
    best_split_val: splitting value resulting lowest entropy value 
    '''
    # loop over all potential splits and calculate the overall entropy of
    # that particular split to find the parameters of best_split_col and
    # best_split_val which generate the lowest overall entropy

    first_iteration = True
    
    # loop over feature(column)
    for column_index in potential_splits:
        
        # loop over potential split value
        for value in potential_splits[column_index]: 
            #getting data_below and data_above for entropy calculation
            data_below, data_above = split_data(data, 
                                                split_column = column_index, 
                                                split_value= value)
    
            if ml_task == "regression":
                current_overall_metric = calculate_overall_metric(data_below,data_above,calculate_mse)

            #calculate local overall entropy
            else:
                
                current_overall_metric = calculate_overall_metric(data_below,data_above,calculate_entropy)

            # check if the current_overall_entropy is smaller than currently stored entropy value
            # in case if 
            if first_iteration == True or current_overall_metric <= overall_metric:
                first_iteration = False
                
                overall_metric = current_overall_metric

                # update split column and value
                best_split_column = column_index
                best_split_value = value

        
            
    return best_split_column, best_split_value


#=================================================================================


# String data = categorical date
def determine_data_type(df):
    data_types = [] 
   
    # number to decide whether the feature is categorical or continous
    n_unique_values_threshold = 15
    
    for column in df.columns:
        unique_values = df[column].unique()
        
        #storing an example data to check whether it is a String type data
        example_value = unique_values[0]
        
        if(isinstance(example_value,str)) or (len(unique_values) <= n_unique_values_threshold):
            data_types.append("categorical")
        else:
            data_types.append("continuous")
        
    return data_types


#=================================================================================

# recursively building sub tree + pruning tree functions
# ml_task decide regresion or classification
def decision_tree_algorithm(df, ml_task, counter = 0, min_sample = 5, max_depth = 3, random_subspace = None):
    '''
    df: Pandas dataframe 
    counter = counts the number of recursive call
    min_sample: min number of sample(data) to split  
    max_depth: max depth of the tree
    '''
    #for data preparation
    if counter == 0:
        
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_data_type(df)
        data = df.values
    else:
        data = df
        
    #base cases: if data is pure or number of data is less than min_sample
    if (check_purity(data) == True) or (len(data) < min_sample) or (counter == max_depth):
        
        #return classification
        leaf = create_leaf(data, ml_task)
        return leaf
    
    # if data is not pure, begin recursive call
    else:
        # since it is not the first call anymore
        counter += 1
        
        #using helper funtion
        potential_splits = get_potential_splits(data,random_subspace)
        
        split_column, split_value = find_best_split(data, potential_splits,ml_task)
        data_below, data_above = split_data(data, split_column, split_value)
        
        
        if len(data_below) == 0 or len(data_above) == 0:
            leaf = create_leaf(data,ml_task)
            return leaf 
        
        # Whole tree is made up with subtrees
        # subtree format: {question : [In case True, In case False]}
        
        #instantiate subtree considering the data type
        
        type_of_feature = FEATURE_TYPES[split_column]
        feature_name = COLUMN_HEADERS[split_column]
        
        if type_of_feature == "continuous":        
            question = "{} <= {}".format(COLUMN_HEADERS[split_column], split_value)
        
        else:
            question = "{} == {}".format(COLUMN_HEADERS[split_column], split_value)
        
        sub_tree = {question: []}
        
        #find True & False case : recursion 
        true_case = decision_tree_algorithm(data_below,ml_task,counter,min_sample,max_depth,random_subspace)
        false_case = decision_tree_algorithm(data_above,ml_task,counter,min_sample,max_depth,random_subspace)
        
        # in case max_detph is reached and the true case and false case return the same result
        
        if true_case == false_case:
            sub_tree = true_case
        else:
            sub_tree[question].append(true_case)
            sub_tree[question].append(false_case)
        
        return sub_tree
        
        


#=================================================================================

# use the tree to make prediction(classify) the instance
# recursion until the instance reach to the class (not sub question)

def classify_instance(instance, tree):
    question = list(tree.keys())[0]
    
    feature_name, comparsion_operator, value = question.split()
    
    # in case continuous
    if comparsion_operator == "<=":

        if instance[feature_name] <= float(value):
            ans = tree[question][0]
        else:
            ans = tree[question][1]
    
    # in case categorical
    else:
        if str(instance[feature_name]) == value:
            ans = tree[question][0]
        else:
            ans = tree[question][1]
    
    # base case: check data type
    if isinstance(ans,dict) == False:
        return ans

    # if did not reach the class = ans is dictionary(sub-tree)
    else: 
        return classify_instance(instance, ans)

#=================================================================================

def calculate_accuracy(df, tree):
    
    # create column of predictions by applying the classify_instance function
    # along an axis of the df
    df["classification"] = df.apply(classify_instance, 
                                    axis = 1,
                                    args = (tree,))
    
    df["classification_result"] = df.classification == df.label
    
    accuracy = df.classification_result.mean()
    
    return accuracy


#=================================================================================

def predict_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return predict_example(example, residual_tree)

#=================================================================================

def decision_tree_predictions(test_df, tree):
    predictions = test_df.apply(predict_example, args = (tree,), axis = 1)
    return predictions



import pandas as pd
import random


# Given with df and test_size(porportion) as arguments, split the
# original df into two parts by randomly selecting indices 

def train_test_split(df, test_size):
    test_size = int(len(df) * test_size)

    # indices contain all the index values of the DF
    indices = df.index.tolist()

    # Chooses k unique random elements from a population sequence or set for test indices 
    test_indices = random.sample(population=indices, k = test_size)

    # loc[] allows us to acces certain rows
    test_df = df.loc[test_indices]

    # Create test_df by dropping test_indices(rows) from original DF
    train_df = df.drop(test_indices)

    return train_df, test_df
    


# String data = categorical date
def determine_type_of_feature(df):
    data_types = [] 
   
    # number to decide whether the feature is categorical or continous
    n_unique_values_threshold = 15
    
    for column in df.columns:
        unique_values = df[column].unique()
        
        #storing an example data to check whether it is a String type data
        example_value = unique_values[0]
        
        if(isinstance(example_value,str)) or (len(unique_values) <= n_unique_values_threshold):
            data_types.append("categorical")
        else:
            data_types.append("continuous")
        
    return data_types


def calculate_accuracy(predictions, labels):
    predictions_correct = predictions == labels
    accuracy = predictions_correct.mean()
    return accuracy





























